# Resistance/Support AND Candles Patterns

In [94]:
import yfinance as yf
import pandas as pd

# ticker = "EURUSD=X"
ticker = "GOOG"

df = yf.download(ticker, start="2002-01-01", end="2023-01-01", interval='1d')

# df = pd.read_csv("EURUSD_Candlestick_1_D_ASK_05.05.2003-30.06.2021.csv")

#Check if NA values are in data
# df=df[df['Volume']!=0]

# rename the columns to start with lowercase letters
# df.columns = [x.lower() for x in df.columns]

# df.reset_index(drop=True, inplace=True)
# df.isna().sum()
# df.tail()
# df =  pd.to_datetime(df['date'])

# Example OHLC daily data for Google Inc.
from backtesting.test import GOOG

print(GOOG.tail())

print(df.tail())

[*********************100%***********************]  1 of 1 completed
             Open    High     Low   Close   Volume
2013-02-25  802.3  808.41  790.49  790.77  2303900
2013-02-26  795.0  795.95  784.40  790.13  2202500
2013-02-27  794.8  804.75  791.11  799.78  2026100
2013-02-28  801.1  806.99  801.03  801.20  2265800
2013-03-01  797.8  807.14  796.15  806.19  2175400
                 Open       High        Low      Close  Adj Close    Volume
Date                                                                       
2022-12-23  87.620003  90.099998  87.620003  89.809998  89.809998  17815000
2022-12-27  89.309998  89.500000  87.535004  87.930000  87.930000  15470900
2022-12-28  87.500000  88.519997  86.370003  86.459999  86.459999  17879600
2022-12-29  87.029999  89.364998  86.989998  88.949997  88.949997  18280700
2022-12-30  87.364998  88.830002  87.029999  88.730003  88.730003  19190300


# Support and Resistance FUNCTIONS

In [79]:
def support(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.low[i]>df1.low[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.low[i]<df1.low[i-1]):
            return 0
    return 1

def resistance(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.high[i]<df1.high[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.high[i]>df1.high[i-1]):
            return 0
    return 1

In [80]:
length = len(df)
high = list(df['high'])
low = list(df['low'])
close = list(df['close'])
open = list(df['open'])
bodydiff = [0] * length

highdiff = [0] * length
lowdiff = [0] * length
ratio1 = [0] * length
ratio2 = [0] * length

def isEngulfing(l):
    row=l
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001      

    bodydiffmin = 0.002
    if (bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]<close[row-1] and
        open[row]>close[row] and 
        (open[row]-close[row-1])>=-0e-5 and close[row]<open[row-1]): #+0e-5 -5e-5
        return 1

    elif(bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]>close[row-1] and
        open[row]<close[row] and 
        (open[row]-close[row-1])<=+0e-5 and close[row]>open[row-1]):#-0e-5 +5e-5
        return 2
    else:
        return 0
       
def isStar(l):
    bodydiffmin = 0.0020
    row=l
    highdiff[row] = high[row]-max(open[row],close[row])
    lowdiff[row] = min(open[row],close[row])-low[row]
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001
    ratio1[row] = highdiff[row]/bodydiff[row]
    ratio2[row] = lowdiff[row]/bodydiff[row]

    if (ratio1[row]>1 and lowdiff[row]<0.2*highdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]>close[row]):
        return 1
    elif (ratio2[row]>1 and highdiff[row]<0.2*lowdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]<close[row]):
        return 2
    else:
        return 0
    
def closeResistance(l,levels,lim):
    if len(levels)==0:
        return 0
    c1 = abs(df.high[l]-min(levels, key=lambda x:abs(x-df.high[l])))<=lim
    c2 = abs(max(df.open[l],df.close[l])-min(levels, key=lambda x:abs(x-df.high[l])))<=lim
    c3 = min(df.open[l],df.close[l])<min(levels, key=lambda x:abs(x-df.high[l]))
    c4 = df.low[l]<min(levels, key=lambda x:abs(x-df.high[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0
    
def closeSupport(l,levels,lim):
    if len(levels)==0:
        return 0
    c1 = abs(df.low[l]-min(levels, key=lambda x:abs(x-df.low[l])))<=lim
    c2 = abs(min(df.open[l],df.close[l])-min(levels, key=lambda x:abs(x-df.low[l])))<=lim
    c3 = max(df.open[l],df.close[l])>min(levels, key=lambda x:abs(x-df.low[l]))
    c4 = df.high[l]>min(levels, key=lambda x:abs(x-df.low[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0

In [81]:
n1=2
n2=2
backCandles=30
signal = [0] * length

for row in range(backCandles, len(df)-n2):
    ss = []
    rr = []
    for subrow in range(row-backCandles+n1, row+1):
        if support(df, subrow, n1, n2):
            ss.append(df.low[subrow])
        if resistance(df, subrow, n1, n2):
            rr.append(df.high[subrow])
    #!!!! parameters
    if ((isEngulfing(row)==1 or isStar(row)==1) and closeResistance(row, rr, 150e-5) ):#and df.RSI[row]<30
        signal[row] = 1
    elif((isEngulfing(row)==2 or isStar(row)==2) and closeSupport(row, ss, 150e-5)):#and df.RSI[row]>70
        signal[row] = 2
    else:
        signal[row] = 0



In [83]:
df['signal']=signal

In [84]:
df[df['signal']==2].count()

open         59
high         59
low          59
close        59
adj close    59
volume       59
signal       59
dtype: int64

In [85]:
df.columns = ['Local time', 'Open', 'High', 'Low', 'Close', 'Volume', 'signal']
# df=df.iloc[100:200]
df

,Local time,Open,High,Low,Close,Volume,signal
0,2.490664,2.591785,2.390042,2.499133,2.499133,897427216,0
1,2.515820,2.716817,2.503118,2.697639,2.697639,458857488,0
2,2.758411,2.826406,2.716070,2.724787,2.724787,366857939,0
3,2.770615,2.779581,2.579581,2.611960,2.611960,306396159,0
4,2.614201,2.689918,2.587302,2.640104,2.640104,184645512,0
...,...,...,...,...,...,...,...
4620,87.620003,90.099998,87.620003,89.809998,89.809998,17815000,0
4621,89.309998,89.500000,87.535004,87.930000,87.930000,15470900,0
4622,87.500000,88.519997,86.370003,86.459999,86.459999,17879600,0
4623,87.029999,89.364998,86.989998,88.949997,88.949997,18280700,0


In [86]:
def SIGNAL():
    return df.signal

In [75]:
#A new strategy needs to extend Strategy class and override its two abstract methods: init() and next().
#Method init() is invoked before the strategy is run. Within it, one ideally precomputes in efficient, 
#vectorized manner whatever indicators and signals the strategy depends on.
#Method next() is then iteratively called by the Backtest instance, once for each data point (data frame row), 
#simulating the incremental availability of each new full candlestick bar.

#Note, backtesting.py cannot make decisions / trades within candlesticks — any new orders are executed on the
#next candle's open (or the current candle's close if trade_on_close=True). 
#If you find yourself wishing to trade within candlesticks (e.g. daytrading), you instead need to begin 
#with more fine-grained (e.g. hourly) data.

In [76]:
from backtesting import Strategy

class MyCandlesStrat(Strategy):  
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next() 
        if self.signal1==2:
            sl1 = self.data.Close[-1] - 750e-4
            tp1 = self.data.Close[-1] + 600e-4
            self.buy(sl=sl1, tp=tp1)
        elif self.signal1==1:
            sl1 = self.data.Close[-1] + 750e-4
            tp1 = self.data.Close[-1] - 600e-4
            self.sell(sl=sl1, tp=tp1)

In [77]:
from backtesting import Backtest
import yfinance as yf
import pandas as pd



bt = Backtest(df, MyCandlesStrat, cash=10_000, commission=.002)
stat = bt.run()
stat

ValueError: Short orders require: TP (32.74908584594726) < LIMIT (32.74346767425537) < SL (32.88408584594727)

In [ ]:
bt.plot()

In [65]:
df

,Open,High,Low,Close,Volume,signal
Local time,,,,,,
1970-01-01 09:27:00.001029968,0.416071,0.392143,0.416071,0.354126,529496800,0
1970-01-01 09:51:25.689640045,0.424107,0.406607,0.421071,0.358381,612007200,0
1970-01-01 10:00:10.309123992,0.427679,0.410536,0.423036,0.360054,409976000,0
1970-01-01 10:09:56.533584594,0.428571,0.406250,0.408929,0.348047,444584000,0
1970-01-01 09:45:00.000000000,0.411607,0.401071,0.403750,0.343639,450038400,0
...,...,...,...,...,...,...
1970-05-11 22:04:47.841796876,132.419998,129.639999,131.860001,131.658981,63814900,0
1970-05-12 09:07:12.421875000,131.410004,128.720001,130.029999,129.831772,69007800,0
1970-05-10 16:04:47.841796876,131.029999,125.870003,126.040001,125.847855,85438400,0
